In [ ]:
import pandas as pd 
import datetime

In [ ]:
# Database related all the code is moved to this file
%run -i "SqlConn.py"
#importing local py file
import SqlConn 

In [ ]:
#Fetch Tweets and Sentiment data - Join from Tweet and Sentiment tables
try:
    df_dbSentimentData = SqlConn.fetchTweetSentimentForStandardizing("")
    print("Data transfer Done")
except BaseException as err:
    print(err)


In [ ]:
#Fetch Tweets and Sentiment data - Join from Tweet and Sentiment tables
try:
    df_dbFinaceData = SqlConn.fetchFinanceData("")
    print("Data transfer Done")
except BaseException as err:
    print(err)


In [ ]:
tweets_full = df_dbSentimentData
tweets_full

In [ ]:
tweets_full['tweetID'].value_counts()

In [ ]:
tweets_full[(tweets_full['tweetID'] == 1022430321696858112)]

In [ ]:
tweets_full.dtypes

In [ ]:
tweets_full['date'] = pd.to_datetime(tweets_full['date'])
tweets_full = tweets_full.sort_values(['date'], ascending = True)
tweets_full

In [ ]:
tweets_full['date'] = tweets_full['date'].dt.date
tweets_full['date']

In [ ]:
tweets_full.set_index('tweetID', inplace=True)
tweets_full

In [ ]:
tweets_filtered_twitter_df = tweets_full[(tweets_full["financeType"] == "twitter")]

In [ ]:
tweets_filtered_tesla_df = tweets_full[(tweets_full["financeType"] == "tesla")]

In [ ]:
tweets_filtered_doge_df = tweets_full[(tweets_full["financeType"] == "doge")]

In [ ]:
tweet_dates_twitter = pd.to_datetime(tweets_filtered_twitter_df['date'])
tweet_dates_twitter

In [ ]:
tweet_dates_tesla = pd.to_datetime(tweets_filtered_tesla_df['date'])
tweet_dates_tesla

In [ ]:
tweet_dates_doge = pd.to_datetime(tweets_filtered_doge_df['date'])
tweet_dates_doge

In [ ]:
Stock_Data = df_dbFinaceData
Stock_Data

In [ ]:
Stock_Data_Twitter_df = Stock_Data[(Stock_Data["financeType"] == "twitter")]
Stock_Data_Twitter_df.reset_index(inplace=True)
Stock_Data_Twitter_df

In [ ]:
Stock_Data_Tesla_df = Stock_Data[(Stock_Data["financeType"] == "tesla")]
Stock_Data_Tesla_df.reset_index(inplace=True)
Stock_Data_Tesla_df

In [ ]:
Stock_Data_Doge_df = Stock_Data[(Stock_Data["financeType"] == "doge")]
Stock_Data_Doge_df.reset_index(inplace=True)
Stock_Data_Doge_df

In [ ]:
Stock_Data_Twitter_df.dtypes

In [ ]:
Stock_Data_Tesla_df.dtypes

In [ ]:
Stock_Data_Doge_df.dtypes

In [ ]:
Stock_Data_Twitter_df['date'] =  pd.to_datetime(Stock_Data_Twitter_df['date'])
Stock_Data_Tesla_df['date'] =  pd.to_datetime(Stock_Data_Tesla_df['date'])
Stock_Data_Doge_df['date'] =  pd.to_datetime(Stock_Data_Doge_df['date'])

In [ ]:
Stock_Data_Doge_df.dtypes

In [ ]:
def finance_tweet_relationship(tweet_dates,finance_stock_df):


    stock_price_per_tweet = {}
    start_date_dic = {}
    
    finance_tweets_df = pd.DataFrame()
    Dates_df = []
    Adj_Close_df = []
    Volume_df = []
    finance_ids_df = []
    Date_Position_df = []
    tweet_ids_df = []
    tweet_date_df = []
        
    

    for tweet_id in tweet_dates.index:
            start_date = tweet_dates[tweet_id] - datetime.timedelta(days=1)

            print (f'working on {tweet_id} with start date {start_date}')

            for i in range(len(finance_stock_df['date'])):

                Dates = []
                Adj_Close = []
                Volume = []
                df = pd.DataFrame()

                if start_date == finance_stock_df['date'][i]:
                    j=i
                    n=0
                    
                    end_date = start_date + datetime.timedelta(days=7)
                    print(f'this is my end date {end_date}')

                    while finance_stock_df['date'][j]<=end_date and j <= (len(finance_stock_df['date'])-2):

                        Dates.append(finance_stock_df['date'][j])
                        Adj_Close.append(finance_stock_df['adjustedClose'][j])
                        Volume.append(finance_stock_df['volume'][j])
                        
                        
                        Dates_df.append(finance_stock_df['date'][j])
                        Adj_Close_df.append(finance_stock_df['adjustedClose'][j])
                        Volume_df.append(finance_stock_df['volume'][j])
                        finance_ids_df.append(finance_stock_df['id'][j])
                        Date_Position_df.append(n)
                        tweet_date_df.append(tweet_dates[tweet_id])
                        tweet_ids_df.append(tweet_id) 
                                            
                        
                        j+=1
                        n+=1
                        
                    df = pd.DataFrame(
                        {'date':Dates,
                         'adjustedClose': Adj_Close,
                         'volume':Volume}
                    ) 


                    stock_price_per_tweet[tweet_id] = df
                    start_date_dic[tweet_id] = start_date

                    print (df)

                    break

                elif start_date < finance_stock_df['date'][i]:

                    start_date = finance_stock_df['date'][i-1]
                    print (f'start date changed to {start_date}')
                    
                    j=i-1
                    n=0
                    
                    end_date = start_date + datetime.timedelta(days=7)
                    print(f'this is my new end date {end_date}')

                    while finance_stock_df['date'][j]<=end_date and j <= (len(finance_stock_df['date'])-2) :
                        
                        Dates.append(finance_stock_df['date'][j])
                        Adj_Close.append(finance_stock_df['adjustedClose'][j])
                        Volume.append(finance_stock_df['volume'][j])
                        
                        Dates_df.append(finance_stock_df['date'][j])
                        Adj_Close_df.append(finance_stock_df['adjustedClose'][j])
                        Volume_df.append(finance_stock_df['volume'][j])
                        finance_ids_df.append(finance_stock_df['id'][j])
                        Date_Position_df.append(n)
                        tweet_date_df.append(tweet_dates[tweet_id])
                        tweet_ids_df.append(tweet_id) 
                                             
                        
                        j+=1
                        n+=1
                        
                    df = pd.DataFrame(
                        {'date':Dates,
                         'adjustedClose': Adj_Close,
                         'volume':Volume}
                    ) 

                    stock_price_per_tweet[tweet_id] = df
                    start_date_dic[tweet_id] = start_date
                    
                    start_date=0

                    print (df)

                    break

                else: 
                    print(f'I am in {i}')
        
    finance_tweets_df = pd.DataFrame(
            {'tweetID':tweet_ids_df,
             'tweetDate':tweet_date_df,
             
             'financeID':finance_ids_df,
             'financeDate':Dates_df,
             'adjustedClose': Adj_Close_df,
             'volume':Volume_df,
             'datePosition':Date_Position_df}
        
    )
                    
    return stock_price_per_tweet, finance_tweets_df, start_date_dic

In [ ]:
stock_price_per_tweet_twitter, twitter_tweets_finance_dates_df, start_date_dic_twitter = finance_tweet_relationship(tweet_dates_twitter,Stock_Data_Twitter_df)

In [ ]:
stock_price_per_tweet_tesla, tesla_tweets_finance_dates_df, start_date_dic_tesla = finance_tweet_relationship(tweet_dates_tesla,Stock_Data_Tesla_df)

In [ ]:
stock_price_per_tweet_doge, doge_tweets_finance_dates_df, start_date_dic_doge = finance_tweet_relationship(tweet_dates_doge,Stock_Data_Doge_df)

In [ ]:
def deltas (df):
    Delta_Adj_Close = []
    Percent_Adj_Close=[]
    Delta_Volume = []
    Percent_Volume = []
    df_delta = pd.DataFrame()
    
    for i in range(len(df['date'])-1):
        
        Delta_Adj_Close.append(df['adjustedClose'][i+1]-df['adjustedClose'][0])
        Percent_Adj_Close.append(((df['adjustedClose'][i+1]-df['adjustedClose'][0])/df['adjustedClose'][0])*100)
        Delta_Volume.append(df['volume'][i+1]-df['volume'][0])
        Percent_Volume.append(((df['volume'][i+1]-df['volume'][0])/df['volume'][0])*100)
        
    df_delta = pd.DataFrame(
            {'Delta Adj Close':Delta_Adj_Close,
             'Percent Adj Close':Percent_Adj_Close,
             'Delta Volume': Delta_Volume,
             'Percent Volume':Percent_Volume}
                ) 

    
    return (df_delta)
    

In [ ]:
delta_per_tweet_twitter = {}

for key in stock_price_per_tweet_twitter.keys():
    
    delta_per_tweet_twitter[key]= deltas(stock_price_per_tweet_twitter[key])

In [ ]:
delta_per_tweet_tesla = {}

for key in stock_price_per_tweet_tesla.keys():
    
    delta_per_tweet_tesla[key]= deltas(stock_price_per_tweet_tesla[key])

In [ ]:
delta_per_tweet_tesla

In [ ]:
delta_per_tweet_doge = {}

for key in stock_price_per_tweet_doge.keys():
    
    delta_per_tweet_doge[key]= deltas(stock_price_per_tweet_doge[key])

In [ ]:
def delta_toDataFrame(delta_per_tweet):   
    tweet_id = []

    delta_adj_0 = []
    delta_adj_1 = []
    delta_adj_2 = []
    delta_adj_3 = []
    delta_adj_4 = []
    delta_adj_5 = []
    delta_adj_6 = []
    delta_adj_7 = []

    per_adj_0 = []
    per_adj_1 = []
    per_adj_2 = []
    per_adj_3 = []
    per_adj_4 = []
    per_adj_5 = []
    per_adj_6 = []
    per_adj_7 = []

    delta_vol_0 = []
    delta_vol_1 = []
    delta_vol_2 = []
    delta_vol_3 = []
    delta_vol_4 = []
    delta_vol_5 = []
    delta_vol_6 = []
    delta_vol_7 = []

    per_vol_0 = []
    per_vol_1 = []
    per_vol_2 = []
    per_vol_3 = []
    per_vol_4 = []
    per_vol_5 = []
    per_vol_6 = []
    per_vol_7 = []

    for key in delta_per_tweet.keys():

        tweet_id.append(key)
        
        if delta_per_tweet[key].empty:

            delta_adj_0.append(0)
            per_adj_0.append(0)
            delta_vol_0.append(0)
            per_vol_0.append(0)

            delta_adj_1.append(0)
            per_adj_1.append(0)
            delta_vol_1.append(0)
            per_vol_1.append(0)


            delta_adj_2.append(0)
            per_adj_2.append(0)
            delta_vol_2.append(0)
            per_vol_2.append(0)       
        
            delta_adj_3.append(0)
            per_adj_3.append(0)
            delta_vol_3.append(0)
            per_vol_3.append(0)
            
            delta_adj_4.append(0)
            per_adj_4.append(0)
            delta_vol_4.append(0)
            per_vol_4.append(0)
            
            delta_adj_5.append(0)
            per_adj_5.append(0)
            delta_vol_5.append(0)
            per_vol_5.append(0)
            
            delta_adj_6.append(0)
            per_adj_6.append(0)
            delta_vol_6.append(0)
            per_vol_6.append(0)
            
            delta_adj_7.append(0)
            per_adj_7.append(0)
            delta_vol_7.append(0)
            per_vol_7.append(0)            
        
        else:

            delta_adj_0.append(delta_per_tweet[key]['Delta Adj Close'][0])
            per_adj_0.append(delta_per_tweet[key]['Percent Adj Close'][0])
            delta_vol_0.append(delta_per_tweet[key]['Delta Volume'][0])
            per_vol_0.append(delta_per_tweet[key]['Percent Volume'][0])

            delta_adj_1.append(delta_per_tweet[key]['Delta Adj Close'][1])
            per_adj_1.append(delta_per_tweet[key]['Percent Adj Close'][1])
            delta_vol_1.append(delta_per_tweet[key]['Delta Volume'][1])
            per_vol_1.append(delta_per_tweet[key]['Percent Volume'][1])

            if len(delta_per_tweet[key]['Delta Adj Close']) >= 3:

                delta_adj_2.append(delta_per_tweet[key]['Delta Adj Close'][2])
                per_adj_2.append(delta_per_tweet[key]['Percent Adj Close'][2])
                delta_vol_2.append(delta_per_tweet[key]['Delta Volume'][2])
                per_vol_2.append(delta_per_tweet[key]['Percent Volume'][2])

                if len(delta_per_tweet[key]['Delta Adj Close']) >= 4:

                    delta_adj_3.append(delta_per_tweet[key]['Delta Adj Close'][3])
                    per_adj_3.append(delta_per_tweet[key]['Percent Adj Close'][3])
                    delta_vol_3.append(delta_per_tweet[key]['Delta Volume'][3])
                    per_vol_3.append(delta_per_tweet[key]['Percent Volume'][3])

                    if len(delta_per_tweet[key]['Delta Adj Close']) >= 5:

                        delta_adj_4.append(delta_per_tweet[key]['Delta Adj Close'][4])
                        per_adj_4.append(delta_per_tweet[key]['Percent Adj Close'][4])
                        delta_vol_4.append(delta_per_tweet[key]['Delta Volume'][4])
                        per_vol_4.append(delta_per_tweet[key]['Percent Volume'][4])

                        if len(delta_per_tweet[key]['Delta Adj Close']) >= 6:

                            delta_adj_5.append(delta_per_tweet[key]['Delta Adj Close'][5])
                            per_adj_5.append(delta_per_tweet[key]['Percent Adj Close'][5])
                            delta_vol_5.append(delta_per_tweet[key]['Delta Volume'][5])
                            per_vol_5.append(delta_per_tweet[key]['Percent Volume'][5])

                            if len(delta_per_tweet[key]['Delta Adj Close']) >= 7:

                                delta_adj_6.append(delta_per_tweet[key]['Delta Adj Close'][6])
                                per_adj_6.append(delta_per_tweet[key]['Percent Adj Close'][6])
                                delta_vol_6.append(delta_per_tweet[key]['Delta Volume'][6])
                                per_vol_6.append(delta_per_tweet[key]['Percent Volume'][6])

                                if len(delta_per_tweet[key]['Delta Adj Close']) >= 8:

                                    delta_adj_7.append(delta_per_tweet[key]['Delta Adj Close'][7])
                                    per_adj_7.append(delta_per_tweet[key]['Percent Adj Close'][7])
                                    delta_vol_7.append(delta_per_tweet[key]['Delta Volume'][7])
                                    per_vol_7.append(delta_per_tweet[key]['Percent Volume'][7])

                                else:
                                    delta_adj_7.append(0)
                                    per_adj_7.append(0)
                                    delta_vol_7.append(0)
                                    per_vol_7.append(0)

                            else:

                                delta_adj_7.append(0)
                                per_adj_7.append(0)
                                delta_vol_7.append(0)
                                per_vol_7.append(0)

                                delta_adj_6.append(0)
                                per_adj_6.append(0)
                                delta_vol_6.append(0)
                                per_vol_6.append(0) 

                        else:

                            delta_adj_7.append(0)
                            per_adj_7.append(0)
                            delta_vol_7.append(0)
                            per_vol_7.append(0)

                            delta_adj_6.append(0)
                            per_adj_6.append(0)
                            delta_vol_6.append(0)
                            per_vol_6.append(0) 

                            delta_adj_5.append(0)
                            per_adj_5.append(0)
                            delta_vol_5.append(0)
                            per_vol_5.append(0)                     

                    else:

                        delta_adj_7.append(0)
                        per_adj_7.append(0)
                        delta_vol_7.append(0)
                        per_vol_7.append(0)

                        delta_adj_6.append(0)
                        per_adj_6.append(0)
                        delta_vol_6.append(0)
                        per_vol_6.append(0) 

                        delta_adj_5.append(0)
                        per_adj_5.append(0)
                        delta_vol_5.append(0)
                        per_vol_5.append(0) 

                        delta_adj_4.append(0)
                        per_adj_4.append(0)
                        delta_vol_4.append(0)
                        per_vol_4.append(0)

                else:

                    delta_adj_7.append(0)
                    per_adj_7.append(0)
                    delta_vol_7.append(0)
                    per_vol_7.append(0)

                    delta_adj_6.append(0)
                    per_adj_6.append(0)
                    delta_vol_6.append(0)
                    per_vol_6.append(0) 

                    delta_adj_5.append(0)
                    per_adj_5.append(0)
                    delta_vol_5.append(0)
                    per_vol_5.append(0)

                    delta_adj_4.append(0)
                    per_adj_4.append(0)
                    delta_vol_4.append(0)
                    per_vol_4.append(0)                

                    delta_adj_3.append(0)
                    per_adj_3.append(0)
                    delta_vol_3.append(0)
                    per_vol_3.append(0)

            else:

                delta_adj_7.append(0)
                per_adj_7.append(0)
                delta_vol_7.append(0)
                per_vol_7.append(0)

                delta_adj_6.append(0)
                per_adj_6.append(0)
                delta_vol_6.append(0)
                per_vol_6.append(0) 

                delta_adj_5.append(0)
                per_adj_5.append(0)
                delta_vol_5.append(0)
                per_vol_5.append(0)

                delta_adj_4.append(0)
                per_adj_4.append(0)
                delta_vol_4.append(0)
                per_vol_4.append(0)                

                delta_adj_3.append(0)
                per_adj_3.append(0)
                delta_vol_3.append(0)
                per_vol_3.append(0)
                
                delta_adj_2.append(0)
                per_adj_2.append(0)
                delta_vol_2.append(0)
                per_vol_2.append(0)



    deltas_df = pd.DataFrame(
                    {
                        'tweetID':tweet_id,

                        'deltaPrice_0':delta_adj_0,
                        'deltaPrice_1':delta_adj_1,
                        'deltaPrice_2':delta_adj_2,
                        'deltaPrice_3':delta_adj_3,
                        'deltaPrice_4':delta_adj_4,
                        'deltaPrice_5':delta_adj_5,
                        'deltaPrice_6':delta_adj_6,
                        'deltaPrice_7':delta_adj_7,                        

                        'percentPrice_0':per_adj_0,
                        'percentPrice_1':per_adj_1,
                        'percentPrice_2':per_adj_2,
                        'percentPrice_3':per_adj_3,
                        'percentPrice_4':per_adj_4,
                        'percentPrice_5':per_adj_5,
                        'percentPrice_6':per_adj_6,
                        'percentPrice_7':per_adj_7,                        

                        'deltaVol_0':delta_vol_0,
                        'deltaVol_1':delta_vol_1,
                        'deltaVol_2':delta_vol_2,
                        'deltaVol_3':delta_vol_3,
                        'deltaVol_4':delta_vol_4,
                        'deltaVol_5':delta_vol_5,
                        'deltaVol_6':delta_vol_6,
                        'deltaVol_7':delta_vol_7,                        

                        'percentVol_0':per_vol_0,
                        'percentVol_1':per_vol_1,
                        'percentVol_2':per_vol_2,
                        'percentVol_3':per_vol_3,
                        'percentVol_4':per_vol_4,
                        'percentVol_5':per_vol_5,
                        'percentVol_6':per_vol_6,
                        'percentVol_7':per_vol_7                        

                    })
    
    return deltas_df


In [ ]:
deltas_twitter_df = delta_toDataFrame(delta_per_tweet_twitter) 
deltas_twitter_df

In [ ]:
deltas_tesla_df = delta_toDataFrame(delta_per_tweet_tesla)
deltas_tesla_df

In [ ]:
deltas_doge_df = delta_toDataFrame(delta_per_tweet_doge)   

In [ ]:
def start_dateToFrame(start_date_dic):    
    tweet_id = []
    start_date = []

    for key in start_date_dic.keys():
        tweet_id.append(key)
        start_date.append(start_date_dic[key])

    start_date_df = pd.DataFrame(
                {'tweetID':tweet_id,
                 'startDate':start_date}
                    ) 

    return start_date_df  

In [ ]:
start_date_twitter_df = start_dateToFrame(start_date_dic_twitter)

In [ ]:
start_date_tesla_df = start_dateToFrame(start_date_dic_tesla)

In [ ]:
start_date_doge_df = start_dateToFrame(start_date_dic_doge)

In [ ]:
start_date_twitter_df.set_index('tweetID', inplace=True)
start_date_tesla_df.set_index('tweetID', inplace=True)
start_date_doge_df.set_index('tweetID', inplace=True)

In [ ]:
deltas_twitter_df.set_index('tweetID', inplace=True)
deltas_tesla_df.set_index('tweetID', inplace=True)
deltas_doge_df.set_index('tweetID', inplace=True)

In [ ]:
int_twitter_df = tweets_filtered_twitter_df.join(start_date_twitter_df)
int_tesla_df = tweets_filtered_tesla_df.join(start_date_tesla_df)
int_doge_df = tweets_filtered_doge_df.join(start_date_doge_df)

In [ ]:
final_twitter_df = int_twitter_df.join(deltas_twitter_df)
final_tesla_df = int_tesla_df.join(deltas_tesla_df)
final_doge_df = int_doge_df.join(deltas_doge_df)

In [ ]:
final_twitter_df['date'] = pd.to_datetime(final_twitter_df['date'])
final_tesla_df['date'] = pd.to_datetime(final_tesla_df['date'])
final_doge_df['date'] = pd.to_datetime(final_doge_df['date'])


In [ ]:
final_twitter_df['datesDifference'] = final_twitter_df['date']-final_twitter_df['startDate']
final_tesla_df['datesDifference'] = final_tesla_df['date']-final_tesla_df['startDate']
final_doge_df['datesDifference'] = final_doge_df['date']-final_doge_df['startDate']

In [ ]:
final_twitter_df['weekendOrHoliday'] = (final_twitter_df['datesDifference']>'1d')
final_tesla_df['weekendOrHoliday'] = (final_tesla_df['datesDifference']>'1d')
final_doge_df['weekendOrHoliday'] = (final_doge_df['datesDifference']>'1d')

In [ ]:
final_tesla_df['financeType']='tesla'
final_twitter_df['financeType']='twitter'
final_doge_df['financeType']='doge'

df_deltas_tostore = pd.concat([final_tesla_df, final_twitter_df, final_doge_df],axis=0,ignore_index=False)

In [ ]:
df_deltas_tostore.reset_index(inplace=True)
df_deltas_tostore

In [ ]:
df_deltas_tostore = df_deltas_tostore.drop('replyCount', axis=1)

In [ ]:
df_deltas_tostore['datesDifference'] = df_deltas_tostore['datesDifference'].dt.days.astype('int16')

In [ ]:
df_deltas_tostore.dtypes

In [ ]:
#df_deltas_tostore.to_csv('deltas_tostore.csv')

In [ ]:
tesla_tweets_finance_dates_df['financeType']='tesla'
twitter_tweets_finance_dates_df['financeType']='twitter'
doge_tweets_finance_dates_df['financeType']='doge'

df_financetweet_tostore = pd.concat([tesla_tweets_finance_dates_df,twitter_tweets_finance_dates_df,doge_tweets_finance_dates_df],axis=0,ignore_index=False)

In [ ]:
df_financetweet_tostore.dtypes

In [ ]:
#df_financetweet_tostore.to_csv('financetweet_tostore.csv')

In [ ]:
# Save Tweet data

try:
    SqlConn.insertDeltas(df_deltas_tostore)
    SqlConn.insertFinanceTweetForecast(df_financetweet_tostore)
    print("Data Transfer Done")
except BaseException as err:
    print(err)

In [ ]:
final_twitter_df['startDate'].value_counts()

In [ ]:
final_tesla_df['startDate'].value_counts()

In [ ]:
final_doge_df['startDate'].value_counts()

In [ ]:
#insert startdate groupby code once everything with SQL works

In [ ]:
# model_sum_columns = {
          
#       "likesCount": 'sum',
#       "retweetCount": 'sum',
#       "sentimentScore" : 'sum',
      
#       "tweetID" : 'count',
            
#       "percentPrice_0" : 'mean',
#       "percentPrice_1" : 'mean',
#       "percentPrice_2" : 'mean',
#       "percentPrice_3" : 'mean',
#       "percentPrice_4" : 'mean'
# }


In [ ]:
# model_average_columns = {
          
#       "likesCount": 'mean',
#       "retweetCount": 'mean',
#       "sentimentScore" : 'mean',
      
#       "tweetID" : 'count',
            
#       "percentPrice_0" : 'mean',
#       "percentPrice_1" : 'mean',
#       "percentPrice_2" : 'mean',
#       "percentPrice_3" : 'mean',
#       "percentPrice_4" : 'mean'
# }

In [ ]:
# deltas_Twitter_df = df_deltas_tostore[(df_deltas_tostore["financeType"] == "twitter")]
# deltas_Tesla_df = df_deltas_tostore[(df_deltas_tostore["financeType"] == "tesla")]
# deltas_Doge_df = df_deltas_tostore[(df_deltas_tostore["financeType"] == "doge")]

In [ ]:
# model_twitter_df_average = deltas_Twitter_df.groupby('startDate').agg(model_average_columns)
# model_tesla_df_average = deltas_Tesla_df.groupby('startDate').agg(model_average_columns)
# model_doge_df_average = deltas_Doge_df.groupby('startDate').agg(model_average_columns)


In [ ]:
# model_twitter_df_sum = deltas_Twitter_df.groupby('startDate').agg(model_sum_columns)
# model_tesla_df_sum = deltas_Tesla_df.groupby('startDate').agg(model_sum_columns)
# model_doge_df_sum = deltas_Doge_df.groupby('startDate').agg(model_sum_columns)


In [ ]:
# # Split data into testing and training sets
# X_train, X_test, y_train, y_test = train_test_split(df_AdjustedClose[['adjustedClose']], df_AdjustedClose[['EMA_10']], test_size=.2)
# # Test set
# print(X_test.describe())

In [ ]:
# from sklearn.linear_model import LinearRegression
# from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
# # Create Regression Model
# model = LinearRegression()
# # Train the model
# model.fit(X_train, y_train)
# # Use model to make predictions
# y_pred = model.predict(X_test)

In [ ]:
# from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
# # Printout relevant metrics
# print("Model Coefficients:", model.coef_)
# print("Mean Absolute Error:", mean_absolute_error(y_test, y_pred))
# print("Coefficient of Determination:", r2_score(y_test, y_pred))